In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [24]:
# Definir la arquitectura de la CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        #Canales de entrada, Canales de salida, Tamaño de Kernel, Stride y Padding
        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)
        self.conv2 = nn.Conv2d(16, 64, 3, 1, 1) 
        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512) 
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        #Entrada (batch, channels, ancho, largo)
        # (64, 3, 32, 32)
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2, 2) # No es una capa aprendible. (feature_maps, stride, padding)
        # (64, 16, 16, 16)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2, 2)
        # (64, 64, 8, 8)
        x = torch.relu(self.conv3(x))
        x = self.dropout(x)
        x = torch.max_pool2d(x, 2, 2)
        # (64, 128, 4, 4)
        x = x.view(-1, 128 * 4 * 4)
        # (64, 2048)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [25]:


# Descargar y cargar el conjunto de datos CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomRotation(45),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
#Iterable del dataset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
#Iterable del dataset
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Is Torch Cuda Available? " + str(torch.cuda.is_available()))

Is Torch Cuda Available? True


In [27]:
# Inicializar el modelo, la función de pérdida y el optimizador
model = CNN()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Entrenamiento del modelo
num_epochs = 25
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')



[1,   200] loss: 1.937
[1,   400] loss: 1.663
[1,   600] loss: 1.569
[2,   200] loss: 1.429
[2,   400] loss: 1.384
[2,   600] loss: 1.339
[3,   200] loss: 1.266
[3,   400] loss: 1.223
[3,   600] loss: 1.213
[4,   200] loss: 1.143
[4,   400] loss: 1.131
[4,   600] loss: 1.103
[5,   200] loss: 1.057
[5,   400] loss: 1.046
[5,   600] loss: 1.047
[6,   200] loss: 0.995
[6,   400] loss: 0.988
[6,   600] loss: 0.983
[7,   200] loss: 0.928
[7,   400] loss: 0.939
[7,   600] loss: 0.928
[8,   200] loss: 0.878
[8,   400] loss: 0.889
[8,   600] loss: 0.875
[9,   200] loss: 0.856
[9,   400] loss: 0.854
[9,   600] loss: 0.845
[10,   200] loss: 0.846
[10,   400] loss: 0.794
[10,   600] loss: 0.809
[11,   200] loss: 0.779
[11,   400] loss: 0.772
[11,   600] loss: 0.788
[12,   200] loss: 0.755
[12,   400] loss: 0.742
[12,   600] loss: 0.755
[13,   200] loss: 0.710
[13,   400] loss: 0.730
[13,   600] loss: 0.722
[14,   200] loss: 0.686
[14,   400] loss: 0.702
[14,   600] loss: 0.687
[15,   200] loss: 0

In [28]:
# Evaluación del modelo
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test images: %d %%' % (100 * correct / total))

Accuracy on the test images: 72 %
